# Lab 3 — clustering

In [66]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle
import random
from collections import Counter
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, ResetTool, PanTool, WheelZoomTool, SaveTool
output_notebook()

%matplotlib inline
plt.style.use("ggplot")
%config IPCompleter.use_jedi = False

Loading BokehJS ...

In [67]:
# Reading an object from disk.
with open("tag-coordinates.pickle", "rb") as f:
    movies = pickle.load(f, encoding="utf-8")

In [68]:
movies

{'007': array([-0.00572777, -0.01384393, -0.00123799,  0.00644884, -0.0053712 ]),
 '007 (series)': array([-0.00478981, -0.01047964, -0.00205688,  0.00525682, -0.00320672]),
 '18th century': array([ 0.01246845,  0.00275974, -0.02393061, -0.0149196 , -0.02650479]),
 '1920s': array([ 0.02222845,  0.00502363, -0.01561281, -0.01629804, -0.01557572]),
 '1930s': array([ 0.02910066, -0.00261228, -0.03382993,  0.00911677, -0.02116353]),
 '1950s': array([ 0.02690253, -0.00677291, -0.02174124, -0.0061211 , -0.00335404]),
 '1960s': array([ 0.02076384,  0.00161975, -0.0139423 ,  0.0049476 ,  0.00129858]),
 '1970s': array([ 0.04716156, -0.01306517, -0.01126725,  0.02416436,  0.02337178]),
 '1980s': array([ 0.01610697, -0.03111045, -0.00752143,  0.01527452,  0.05418083]),
 '19th century': array([ 0.01745149,  0.00337408, -0.01974771, -0.01437436, -0.02365988]),
 '3d': array([-9.05875575e-03, -1.93165161e-02, -5.97160916e-03, -3.14385208e-02,
         9.71025760e-05]),
 '70mm': array([ 0.00164807, -0.

In [69]:
def euclide_dist(data,point):
    data = np.asarray(data)
    point = np.asarray(point)
    return np.linalg.norm(data - point)

In [70]:
def initCentroids(k):
    kPoints = []
    for i in range(0,k):
        maxCoord = np.max(G[:,i])
        minCoord = np.min(G[:,i])
        point = [random.uniform(minCoord, maxCoord) for i in range(0,5) ]
        kPoints.append(point)
    return kPoints

In [71]:
def calcError(data_clust, clusters, centroids, dist_fun):
    err = 0
    for i in range(len(clusters)) :
        cluster = clusters[i]
        for point in cluster:
            err += dist_fun(point, centroids[i])
            
            
    return err

In [72]:
def calculateCluster(n, k, centroids, G):
    oldError = 0
    
    while(True):
        clust = [[]for i in range(0,k)]
        data_clust = []
        for data in G:
            euc_dist = []
            for point in centroids:
                euc_dist.append(euclide_dist(data, point))
            minDist = min(euc_dist)
            indexCluster = euc_dist.index(minDist)
            clust[indexCluster].append(data)
            data_clust.append(indexCluster)
        centroids = recalculate_centroids(centroids, clust, k)
        newError = calcError(data_clust,clust, centroids, euclide_dist)
        if(newError == oldError):
            break
        oldError= newError
        
    return data_clust, clust, centroids

In [73]:
def recalculate_centroids(centroids, clusters, k):
    """ Recalculates the centroid position based on the plot """
    clusters = np.asarray(clusters)
    for i in range(k):
        if clusters[i] :
            centroids[i] = np.average(clusters[i], axis=0)
    return centroids

In [74]:
def kMeansAlgo(G, k, n):
    kPoints = initCentroids(k)
    data_clusters, clusters, kPoints = calculateCluster(n, k, kPoints, G)
    return data_clusters, clusters, kPoints

In [75]:
tag = np.asarray(list(movies.keys()))
G = np.asarray(list(movies.values()))
k = 2
n = 50
data_clusters, clusters, kPoints = kMeansAlgo(G, k, n)


clusters = np.asarray(clusters)
data_clusters = np.asarray(data_clusters)
kPoints = np.asarray(kPoints)


In [80]:
from bokeh.palettes import Dark2_8


source = ColumnDataSource(
    data={
        "x": [x[0] for x in G],
        "y": [x[3] for x in G],
        "name": [x for x in tag],
        "color": [Dark2_8[x] for x in data_clusters],
    })

hover = HoverTool(
    tooltips=[
        ("Name", "@name"),
    ])
tools = [hover, ResetTool(), PanTool(), WheelZoomTool(), SaveTool()]

p = figure(plot_width=960, plot_height=360, tools=tools, title="clustering")
p.circle("x", "y", source=source, size=20, color="color", alpha=0.5)
show(p, notebook_handle=True)

In [16]:
tag = np.asarray(list(movies.keys()))
G = np.asarray(list(movies.values()))
k = 3
n = 50
data_clusters, clusters, kPoints = kMeansAlgo(G, k, n)
clusters = np.asarray(clusters)
data_clusters = np.asarray(data_clusters)
kPoints = np.asarray(kPoints)

In [81]:
from bokeh.palettes import Dark2_8


source = ColumnDataSource(
    data={
        "x": [x[0] for x in G],
        "y": [x[3] for x in G],
        "name": [x for x in tag],
        "color": [Dark2_8[x] for x in data_clusters],
    })

hover = HoverTool(
    tooltips=[
        ("Name", "@name"),
    ])
tools = [hover, ResetTool(), PanTool(), WheelZoomTool(), SaveTool()]

p = figure(plot_width=960, plot_height=360, tools=tools, title="clustering")
p.circle("x", "y", source=source, size=20, color="color", alpha=0.5)
show(p, notebook_handle=True)

In [57]:
tag = np.asarray(list(movies.keys()))
G = np.asarray(list(movies.values()))
k = 4
n = 50
data_clusters, clusters, kPoints = kMeansAlgo(G, k, n)
clusters = np.asarray(clusters)
data_clusters = np.asarray(data_clusters)
kPoints = np.asarray(kPoints)

In [82]:
from bokeh.palettes import Dark2_8


source = ColumnDataSource(
    data={
        "x": [x[0] for x in G],
        "y": [x[3] for x in G],
        "name": [x for x in tag],
        "color": [Dark2_8[x] for x in data_clusters],
    })

hover = HoverTool(
    tooltips=[
        ("Name", "@name"),
    ])
tools = [hover, ResetTool(), PanTool(), WheelZoomTool(), SaveTool()]

p = figure(plot_width=960, plot_height=360, tools=tools, title="clustering")
p.circle("x", "y", source=source, size=20, color="color", alpha=0.5)
show(p, notebook_handle=True)

In [29]:
tag = np.asarray(list(movies.keys()))
G = np.asarray(list(movies.values()))
k = 5
n = 50
data_clusters, clusters, kPoints = kMeansAlgo(G, k, n)
clusters = np.asarray(clusters)
data_clusters = np.asarray(data_clusters)
kPoints = np.asarray(kPoints)

In [84]:
from bokeh.palettes import Dark2_8


source = ColumnDataSource(
    data={
        "x": [x[0] for x in G],
        "y": [x[3] for x in G],
        "name": [x for x in tag],
        "color": [Dark2_8[x] for x in data_clusters],
    })

hover = HoverTool(
    tooltips=[
        ("Name", "@name"),
    ])
tools = [hover, ResetTool(), PanTool(), WheelZoomTool(), SaveTool()]

p = figure(plot_width=960, plot_height=360, tools=tools, title="clustering")
p.circle("x", "y", source=source, size=20, color="color", alpha=0.5)
show(p, notebook_handle=True)

The first and fourth direction seem to separate the data quite distincly

In [32]:
# Reading an object from disk.
with open("most-rated.pickle", "rb") as f:
    most_rated = pickle.load(f, encoding="utf-8")
most_rated_movies = list(map(lambda y : y[1],most_rated))
most_rated_movies
len(most_rated_movies)

1000

In [33]:
# Reading an object from disk.
movie_data = sc.textFile("/ix/ml-20m/movies.txt").map(json.loads).filter(lambda y : y["title"] in most_rated_movies)
movie_data = movie_data.collect()

In [34]:
def jaccard_index(movie_a,movie_b):
    category_intersection = [value for value in movie_a['genres'] if value in movie_b['genres']]
    intersection = len(set(movie_a['genres']) & set(movie_b['genres']))
    union = len(set(movie_a['genres']).union(set(movie_b['genres'])))
    return float(intersection)/float(union)

In [45]:
def k_medioids(points, dist_func, k):
    medioids = random.sample(points, k)
    old_medioids=None
    clusters = [[]]
    old_clusters = None
    #after the 
    for i in range(6):
        print(medioids)
        old_medioids = medioids.copy()
        
        old_clusters = clusters.copy()

        clusters = [[] for i in range(k)]
        for point in points:
            dist_to_medioids = []
            for medioid in medioids:
                dist_to_medioids.append(dist_func(medioid, point))
            closest_medioid = min(dist_to_medioids)
            cm_index = dist_to_medioids.index(closest_medioid)
            clusters[cm_index].append(point)
        for i in range(k):
            dists = []
            for point_x in clusters[i]:
                point_distance_sum = 0
                for point_y in clusters[i]:
                    point_distance_sum += dist_func(point_x, point_y)
                dists.append(point_distance_sum)
            min_dist_index = dists.index(min(dists))
            medioids[i] = clusters[i][min_dist_index]
    return clusters, old_clusters

    
            

In [53]:
results_one, results_two = k_medioids(movie_data, jaccard_index, 2)
category_count_one = [[item['genres'] for item in cluster] for cluster in results_one]

category_count_two = [[item['genres'] for item in cluster] for cluster in results_two]

print(len(category_count_one[0]))
print(len(category_count_two[0]))

[{'genres': ['Action', 'Adventure', 'Thriller'], 'movieId': 5872, 'title': 'Die Another Day (2002)'}, {'genres': ['Comedy'], 'movieId': 342, 'title': "Muriel's Wedding (1994)"}]
[{'genres': ['Documentary'], 'movieId': 162, 'title': 'Crumb (1994)'}, {'genres': ['Adventure', 'Animation', 'Children', 'Fantasy', 'Musical'], 'movieId': 661, 'title': 'James and the Giant Peach (1996)'}]
[{'genres': ['Western'], 'movieId': 383, 'title': 'Wyatt Earp (1994)'}, {'genres': ['Comedy', 'Documentary', 'Drama'], 'movieId': 8464, 'title': 'Super Size Me (2004)'}]
980
997


We can see that the medioids that are being printed each iteration alternate after a while and therefore the algorithm has converged but as each iteration calculates new medioids the output depends on the parity of iterations. To get a complete picture we visualize both cluster sets, one for odd and one for even number of iterations after convergence.

In [48]:
frequencies_one = [ dict(Counter(i for sub in cluster for i in set(sub))) for cluster in category_count_one]
frequencies_two = [ dict(Counter(i for sub in cluster for i in set(sub))) for cluster in category_count_two]

[{'Comedy': 383, 'Adventure': 222, 'Animation': 63, 'Children': 83, 'Fantasy': 114, 'Romance': 196, 'Action': 280, 'Thriller': 269, 'Crime': 161, 'Drama': 441, 'Horror': 72, 'Mystery': 83, 'Sci-Fi': 168, 'Musical': 44, 'War': 48, 'IMAX': 22, 'Documentary': 6, 'Film-Noir': 13}, {'Western': 23, 'Action': 11, 'Romance': 3, 'Drama': 7, 'War': 2, 'Thriller': 1, 'Comedy': 9, 'Adventure': 7, 'Mystery': 1, 'Sci-Fi': 2}]
[{'Comedy': 391, 'Adventure': 229, 'Animation': 63, 'Children': 83, 'Fantasy': 114, 'Romance': 199, 'Action': 291, 'Thriller': 270, 'Crime': 161, 'Drama': 447, 'Horror': 72, 'Mystery': 84, 'Sci-Fi': 170, 'Musical': 44, 'War': 50, 'IMAX': 22, 'Western': 23, 'Film-Noir': 13}, {'Documentary': 6, 'Drama': 1, 'Comedy': 1}]


In [51]:
for cluster in frequencies_one:
    show_cluster(categories, cluster)

The most noticeable difference between these two clusteres is the western category

In [52]:
for cluster in frequencies_two: 
    show_cluster(categories,cluster)

The most noticeable difference netween the two clusters here is the documentary category, probably because most documentaries are only of the documentary category and therefore have a high jaccard index compared to movies of different categories. 

This is also the case with westerns but to a less noticeable effect. This is why the k-medioids algorithm ends up alternating between separating by documentary or western. In short, the two clusters seem to always contain one big and one small one, with the small one being comprised of mostly one category which has the least in common with the others.
